In [ ]:
import seaborn as sns
print("Seaborn is installed and working!")


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as pyo 
import plotly.io as pio

import plotly.express as px
from plotly.offline import iplot , plot
from plotly.subplots import make_subplots


import warnings
warnings.filterwarnings('ignore')




from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [ ]:
df=pd.read_csv("/Users/macfreaks/Downloads/Renewable_energy.csv")

In [ ]:
df

In [ ]:
df.head()
df.tail()

In [ ]:

df.info()

## Inspection and cleaning 

In [ ]:

df.isnull().sum()

In [ ]:
def summary(df):
    print(f'data shape: {df.shape}')
    summ = pd.DataFrame(df.dtypes, columns=['Data Type'])
    summ['Missing#'] = df.isna().sum()
    summ['Missing%'] = (df.isna().sum())/len(df)
    summ['Dups'] = df.duplicated().sum()
    summ['Uniques'] = df.nunique().values
    summ['Count'] = df.count().values
    desc = pd.DataFrame(df.describe(include='all').transpose())
    summ['Min'] = desc['min'].values
    summ['Max'] = desc['max'].values
    summ['Average'] = desc['mean'].values
    summ['Standard Deviation'] = desc['std'].values
    summ['First Value'] = df.loc[0].values
    summ['Second Value'] = df.loc[1].values
    summ['Third Value'] = df.loc[2].values

    display(summ)

summary(df)

In [ ]:


index_names = df[ df['Year'] == 2024 ].index

df.drop(index_names, inplace = True)

In [ ]:
df.head(10)



## Explainatory Data Analysis and Types of Energy

In [ ]:
cols = ['Hydroelectric Power', 'Geothermal Energy', 'Solar Energy', 'Wind Energy',
       'Waste Energy', 'Fuel Ethanol, Excluding Denaturant', 'Biomass Losses and Co-products',
       'Renewable Diesel Fuel', 'Other Biofuels', 'Conventional Hydroelectric Power',
       'Biodiesel']

sectors = ['Commerical', 'Electric Power', 'Industrial', 'Residential',
           'Transportation']

colors = ["#8c0404","#f25ed0","#21618C","#16A085","#34495E"]

In [ ]:
fig, axes = plt.subplots(11, 1, figsize=(8, 40))

for ind, col in enumerate(cols):
    plt.subplot(11, 1, ind+1)
    df_top = df.groupby(['Year'])[[col]].sum().reset_index()
    df_top = df_top.sort_values(by=(col), ascending=False)

    sns.barplot(x=df_top["Year"], y=df_top[col], palette="Blues")
    plt.yticks(rotation=45, fontsize = 7)
    plt.xticks(rotation=90, fontsize = 7)   
    plt.title(f"Consumption per Year according to {col}", fontsize = 12, color = 'blue')

plt.tight_layout()

In [ ]:
df_year = df.groupby('Year').sum().reset_index().drop(columns=['Month'])
plt.figure(figsize=(12, 6))
for energy_type in cols:
    plt.plot(df_year['Year'], df_year[energy_type], label=energy_type)

plt.xlabel('Year')
plt.ylabel('Energy Consumption')
plt.title(f'Year-on-Year Consumption of Different Types of Energy',  fontsize = 15, color = 'blue')
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))

plt.tight_layout()
plt.show()


In [ ]:
df_year = df.groupby('Year').sum().reset_index().drop(columns=['Month'])

# Plot
df_year.plot(x="Year", y=['Wind Energy', 'Conventional Hydroelectric Power', 'Solar Energy'],
        kind="line", figsize=(10, 6))
plt.title("Wind / Solar vs Conventional Hydroelectric Power", fontsize = 16, color = 'blue')
plt.ylabel('Consumption', rotation=90, fontsize = 10)
plt.annotate('Surpasses',
      horizontalalignment = 'left',
      verticalalignment = 'bottom',
      xytext = (2012, 1200),
      xy = (2018.5, 1000),
      arrowprops = dict(facecolor = 'red', shrink = 0.05))
plt.annotate('Surpasses',
      horizontalalignment = 'left',
      verticalalignment = 'bottom',
      xytext = (2020, 1100),
      xy = (2022.5, 850),
      arrowprops = dict(facecolor = 'yellow', shrink = 0.05))

plt.grid()
plt.show()

In [ ]:
df['Sector'].unique()

In [ ]:
for i in cols:
    
    if i == 'Hydroelectric Power':

        df_top = df.groupby(['Sector'])[['Hydroelectric Power']].sum()#.reset_index()
        df_top = df_top.sort_values(by=("Hydroelectric Power"), ascending=False).head(10)

        iplot(px.bar(df_top[:10],
             text_auto = True,
             color = df_top[:10].index,
             color_discrete_sequence = colors, 
             labels=dict(index="Count Names",value="Count"),
             title = 'Top Sectors acording to Hydroelectric Power Consumption'
            ))   
    
    else:
        
        df_top = df.groupby(['Sector'])[[i]].sum()#.reset_index()
        df_top = df_top.sort_values(by=[i], ascending=False).head()

        iplot(px.bar(df_top[:5],
             text_auto = True,
             color = df_top[:5].index,
             color_discrete_sequence = colors, 
             labels=dict(index="Count Names",value="Count"),
             title = 'Top Sectors acording to ' +i+ ' Consumption'
             ))

In [ ]:
for i in sectors:
    
    if i == 'Commerical':
        df_sec = df[df['Sector'] == 'Commerical']
        df_sec = df_sec.groupby('Year')[cols].sum().reset_index()

        df_sec.plot(x="Year", y=cols, kind="line", figsize=(8, 5))
        plt.title("Energy Consumption by Commerical over the years", fontsize = 14, color = 'blue')
        plt.ylabel('Energy Consumption', rotation=90, fontsize = 10)
        plt.legend(loc='upper left', bbox_to_anchor=(1, 1))

        plt.grid()
        plt.show()
    else:
        df_sec = df[df['Sector'] == i]
        df_sec = df_sec.groupby('Year')[cols].sum().reset_index()

        df_sec.plot(x="Year", y=cols, kind="line", figsize=(8, 5))
        plt.title(f"Energy Consumption by {i} over the years", fontsize = 14, color = 'blue')
        plt.ylabel('Energy Consumption', rotation=90, fontsize = 10)
        plt.legend(loc='upper left', bbox_to_anchor=(1, 1))

        plt.grid()
        plt.show()


In [ ]:
df_2023 = df[df['Year'] == 2023]

plt.figure(figsize=(12, 6))
bottom = np.zeros(len(df_2023))
for source in cols:
    plt.bar(df_2023['Sector'], df_2023[source], bottom=bottom, label=source)
    bottom += df_2023[source]
plt.xlabel('Sector')
plt.ylabel('Energy')
plt.title(f'Energy Sources by Sector in 2023')
plt.legend(bbox_to_anchor=(1.02, 1))
plt.tight_layout()
plt.show()

In [ ]:
df['Total_Energy'] = df['Hydroelectric Power'] + df['Geothermal Energy'] + df['Solar Energy']+ df['Wind Energy'] + df['Wood Energy'] + df['Waste Energy'] + df['Fuel Ethanol, Excluding Denaturant'] + df['Biomass Losses and Co-products'] + df['Biomass Energy'] + df['Renewable Diesel Fuel'] + df['Other Biofuels'] + df['Conventional Hydroelectric Power'] + df['Biodiesel']

In [ ]:
df_com = df[df['Sector'] == 'Commerical']
df_ele = df[df['Sector'] == 'Electric Power']
df_ind = df[df['Sector'] == 'Industrial']
df_res = df[df['Sector'] == 'Residential']
df_tra = df[df['Sector'] == 'Transportation']

# Groupby - Year
df_com = df_com.groupby('Year')['Total_Energy'].sum().reset_index()
df_ele = df_ele.groupby('Year')['Total_Energy'].sum().reset_index()
df_ind = df_ind.groupby('Year')['Total_Energy'].sum().reset_index()
df_res = df_res.groupby('Year')['Total_Energy'].sum().reset_index()
df_tra = df_tra.groupby('Year')['Total_Energy'].sum().reset_index()


In [ ]:
df_com.rename(columns = {'Total_Energy':'Commerical'}, inplace = True)
df_ele.rename(columns = {'Total_Energy':'Electric Power'}, inplace = True)
df_ind.rename(columns = {'Total_Energy':'Industrial'}, inplace = True)
df_res.rename(columns = {'Total_Energy':'Residential'}, inplace = True)
df_tra.rename(columns = {'Total_Energy':'Transportation'}, inplace = True)

# Concat datasets 
total_energy = pd.concat([df_com, df_ele, df_ind, df_res, df_tra])

In [ ]:
total_energy.plot(x="Year", y=['Commerical', 'Electric Power', 'Industrial', 'Residential', 'Transportation'],
              kind="line", figsize=(10, 6))
plt.title("Total Energy consumption by Sector over the years", fontsize = 14, color = 'blue')
plt.ylabel('Total_Energy', rotation=90, fontsize = 10)

plt.grid()
plt.show()

## Correelation Analysis


In [ ]:
df_encoded = pd.get_dummies(df, columns=['Sector'], drop_first=True)
correlation_matrix = df_encoded.drop(columns=['Year', 'Month']).corr()

plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Heatmap between Energy Types and Sectors')
plt.show()

In [ ]:


correlation_matrix 

## Positive Correlation:
Hydroelectric Power shows strong positive correlations with Wood Energy (0.82) and Waste Energy (0.35). This suggests that these energy types tend to be positively correlated with hydroelectric power production.
Wood Energy and Waste Energy also have a positive correlation of 0.36, indicating a tendency for these two energy sources to be positively associated.
Total Renewable Energy has positive correlations with many individual renewable energy types, such as Hydroelectric Power (0.50), Geothermal Energy (0.39), and Biomass Energy (0.61). This suggests that overall renewable energy production tends to be positively correlated with the production of various specific renewable energy sources.


## Negative Correlation:
Geothermal Energy and Wind Energy exhibit a negative correlation (-0.27). This suggests that as production in one of these energy types increases, production in the other tends to decrease.
Solar Energy and Wind Energy also show a negative correlation (-0.15), implying a similar negative relationship between these two energy sources.
Renewable Diesel Fuel and Other Biofuels show a strong positive correlation of 0.94, indicating a high degree of positive association between these two biofuel types.


## Correlation with Sectors:
Electric Power sector has a strong positive correlation with Industrial sector (0.88), suggesting that energy production in these two sectors tends to be positively associated.
The Residential sector has a negative correlation with many energy types, indicating a tendency for lower residential energy consumption of these energy sources compared to other sectors.
Transportation sector shows a positive correlation with Fuel Ethanol and Other Biofuels, which implies a higher usage of biofuels in transportation compared to other sectors.


## Hydroelectric Power and Geothermal Energy:
Hydroelectric Power shows a moderate negative correlation with Geothermal Energy (-0.27). This suggests that in some cases, the production of hydroelectric power may be inversely related to the production of geothermal energy. This could be due to geographical factors or resource allocation decisions.

## Wood Energy and Waste Energy:
Wood Energy exhibits a strong positive correlation with Waste Energy (0.36). This correlation suggests that there may be similar factors influencing the production of both wood and waste energy, such as policies promoting biomass utilization or waste-to-energy initiatives.

## Renewable Diesel Fuel and Other Biofuels:
Renewable Diesel Fuel and Other Biofuels display a remarkably high positive correlation of 0.94. This strong correlation implies that these two types of biofuels often experience similar trends in production. It could be indicative of common market drivers, such as government incentives or technological advancements, impacting both biofuel types equally.


## Sectoral Analysis:
The Electric Power sector exhibits a significant positive correlation with the Industrial sector (0.88). This strong correlation suggests that energy consumption and production patterns in these sectors are closely intertwined. It could indicate shared demand drivers, such as economic activity or industrial expansion, leading to increased energy consumption.
The Transportation sector shows a notable positive correlation with Fuel Ethanol (0.61) and Other Biofuels (0.48). This suggests that biofuels, particularly ethanol and other alternative fuels, play a significant role in the transportation sector, potentially driven by regulatory mandates or environmental initiatives promoting cleaner fuels.

## Conventional Hydroelectric Power:
Conventional Hydroelectric Power exhibits a strong positive correlation with the Electric Power sector (0.98). This correlation underscores the dominant role of hydroelectricity in electricity generation, especially in regions with significant hydroelectric infrastructure.


## Biodiesel:
Biodiesel shows a strong positive correlation with Fuel Ethanol (0.91) and Other Biofuels (0.57). This suggests that biofuel production, including biodiesel, ethanol, and other alternative fuels, may be influenced by common factors such as government policies supporting renewable fuel standards or incentives for biofuel production.


## According to type of electricity consumed


In [ ]:
electricity_consumption_data = df

electricity_consumption_data['Total_Renewable_Energy'] = electricity_consumption_data[['Solar Energy', 'Wind Energy', 'Hydroelectric Power', 'Other Biofuels', 'Conventional Hydroelectric Power']].sum(axis=1)
electricity_consumption_data['Solar_Contribution'] = electricity_consumption_data['Solar Energy'] / electricity_consumption_data['Total_Renewable_Energy']
electricity_consumption_data['Wind_Contribution'] = electricity_consumption_data['Wind Energy'] / electricity_consumption_data['Total_Renewable_Energy']
electricity_consumption_data['Hydro_Contribution'] = electricity_consumption_data['Hydroelectric Power'] / electricity_consumption_data['Total_Renewable_Energy']
electricity_consumption_data['Other_Biofuels_Contribution'] = electricity_consumption_data['Other Biofuels'] / electricity_consumption_data['Total_Renewable_Energy']
electricity_consumption_data['Conventional_Hydroelectric_Power_Contribution'] = electricity_consumption_data['Conventional Hydroelectric Power'] / electricity_consumption_data['Total_Renewable_Energy']

years = electricity_consumption_data['Year'].unique()

plt.figure(figsize=(12, 8))

plt.stackplot(years, 
              electricity_consumption_data.groupby('Year')['Solar_Contribution'].sum(),
              electricity_consumption_data.groupby('Year')['Wind_Contribution'].sum(),
              electricity_consumption_data.groupby('Year')['Hydro_Contribution'].sum(),
              electricity_consumption_data.groupby('Year')['Other_Biofuels_Contribution'].sum(),
              electricity_consumption_data.groupby('Year')['Conventional_Hydroelectric_Power_Contribution'].sum(),
              labels=['Solar', 'Wind', 'Hydro', 'Other Biofuels', 'Conventional Hydroelectric Power'],
              colors=['gold', 'skyblue', 'lightgreen', 'orange', 'lightcoral'])
plt.xlabel('Year')
plt.ylabel('Contribution to Total Renewable Energy Production')
plt.title('Contribution of Different Energy Sources to Total Renewable Energy Production Over Time')
plt.legend(loc='upper left')
plt.xticks(years, rotation=45)
plt.tight_layout()
plt.show()

## Applying Machine Learning Model

In [ ]:
import tensorflow as tf

In [ ]:
import tensorflow as tf

In [ ]:
data = df.drop(columns=['Sector'])
data = data[data['Year'] != 2024]
data_grouped = data.groupby('Year').sum()
data_grouped.drop(columns=['Month'], inplace=True)

In [ ]:
data_grouped.head()

In [ ]:
data_grouped.reset_index(inplace=True)

data_grouped.index = data_grouped.index - data_grouped.index.min()

In [ ]:
data_grouped.drop(columns=[], inplace=True)

In [ ]:
data_grouped.head()

In [ ]:
energy_data = data_grouped

In [ ]:
class SimpleForecastingModel(tf.keras.Model):
    def __init__(self, hidden_size, output_size):
        super(SimpleForecastingModel, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = tf.keras.layers.SimpleRNN(hidden_size, return_sequences=True, return_state=True)
        self.fc = tf.keras.layers.Dense(output_size)
        
    def call(self, inputs):
        out, _ = self.rnn(inputs)
        out = self.fc(out[:, -1, :])
        return out

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Compute correlation matrix
corr_matrix = df.corr()

# Plot heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
plt.title("Correlation Heatmap of Renewable Energy Sources")
plt.show()


In [ ]:
%pip install seaborn matplotlib pandas scikit-learn


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

print("Libraries imported successfully!")



In [ ]:
plt.figure(figsize=(12, 6))
for col in df.columns[1:]:  # Assuming first column is 'Year'
    plt.plot(df['Year'], df[col], label=col)

plt.xlabel("Year")
plt.ylabel("Energy Consumption (TWh)")
plt.title("Renewable Energy Consumption Over Time")
plt.legend()
plt.show()


In [ ]:
df

In [ ]:
df = pd.read_csv("/Users/macfreaks/Downloads/Renewable_energy.csv")
df = df.sort_index()

In [ ]:
df

In [ ]:
df.plot(figsize=(12, 6))
plt.title("Renewable Energy Consumption Over Time")
plt.show()

In [ ]:
global energy_col
energy_col = df.columns[0]  # Select the first column dynamically
print("Using column:", energy_col)


In [ ]:



df.plot(y=energy_col, figsize=(12, 6))
plt.title("Renewable Energy Consumption Over Time")
plt.xlabel("Date")
plt.ylabel("Energy Consumption")
plt.show()


In [ ]:
result = adfuller(df[energy_col].dropna())
print("ADF Test p-value:", result[1])


In [ ]:
# ARIMA Model
model_arima = ARIMA(df[energy_col], order=(5,1,0))
model_fit = model_arima.fit()
print(model_fit.summary())


In [ ]:
df["ARIMA_Pred"] = model_fit.predict(start=0, end=len(df)-1)
plt.figure(figsize=(12,6))
plt.plot(df.index, df[energy_col], label="Actual")
plt.plot(df.index, df["ARIMA_Pred"], label="ARIMA Prediction", linestyle='dashed')
plt.xlabel("Date")
plt.ylabel("Energy Consumption")
plt.legend()
plt.show()


In [ ]:
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df[[energy_col]])

def create_sequences(data, seq_length=10):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

seq_length = 10
X, y = create_sequences(df_scaled, seq_length)
X_train, X_test = X[:-30], X[-30:]
y_train, y_test = y[:-30], y[-30:]

model_lstm = Sequential([
    LSTM(50, return_sequences=True, input_shape=(seq_length, 1)),
    LSTM(50, return_sequences=False),
    Dense(25),
    Dense(1)
])

model_lstm.compile(optimizer='adam', loss='mse')
model_lstm.fit(X_train, y_train, epochs=20, batch_size=16, verbose=1)

y_pred = model_lstm.predict(X_test)
y_pred = scaler.inverse_transform(y_pred)

plt.figure(figsize=(12,6))
plt.plot(df.index[-30:], df[energy_col].iloc[-30:], label="Actual")
plt.plot(df.index[-30:], y_pred, label="LSTM Prediction", linestyle='dashed')
plt.legend()
plt.xlabel("Date")
plt.ylabel("Energy Consumption")
plt.show()
